In [ ]:
using DDA, Plots
gr()

# Material models
[source](http://www.fdtdxx.com/features/material-models)
[article](https://www.researchgate.net/publication/249651487_Free-Space_Transmission_Method_for_the_Characterization_of_Dielectric_and_Magnetic_Materials_at_Microwave_Frequencies)

## Dielectrics and Conductors: 

- dielectric model: This includes perfect (lossless) dielectrics, conductive (lossy) materials, and perfect electric conductors.
$$\varepsilon (\omega) = \varepsilon_\infty + \frac{\sigma}{j \omega \varepsilon_0}$$


## Dispersive Materials

Dispersive (frequency-dependent) materials can correctly describe the physics behind the optical responses. 

- Dielectric constant of free electron gas: 
  $$\varepsilon (\omega) = 1 - \frac{\omega_p^2}{\omega^2}$$
  
- Drude model: describes intraband electron motion.
  $$\varepsilon (\omega) = 1 - \frac{\omega_p^2}{\omega^2 + j \omega \gamma}$$

- Lorentz oscillator model: describes interband electron transitions.
  $$\varepsilon (\omega) = 1 + \frac{\Delta\varepsilon_p \omega_p^2}{\omega_p^2 - \omega(\omega + j 2\delta)}$$

- Debye model: describes the relaxation of atomic vibrations.
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{1 + j \omega \tau}$$
  where 
  - $\Delta\varepsilon =  \varepsilon_\infty - \varepsilon_s$
  - $\tau = \frac{1}{\omega_r}$
  - $\varepsilon_s$ is a static dielectric permittivity,
  - $\varepsilon_\infty$ is permittivity at infinite frequency (optical permittivity), 
  - $\omega_r$ is relaxation frequency.

- Cole–Cole model 
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{1 + j (\omega \tau)^{(1-\alpha)}} $$

- Cole-Davidson
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{(1 + j \omega \tau)^\beta} $$
  
- Havriliak-Negami
  $$\varepsilon (\omega) = \varepsilon_\infty  + \frac{\Delta\varepsilon}{(1 + j (\omega \tau)^{(1-\alpha)})^\beta} $$

The terms $\alpha$ and $\beta$ are empirical parameters and their values are between $0$ and $1$. 
- $\alpha$ is a damping factor (broaden the dispersion) and describes the degree of flatness of the relaxation region. 
- $\beta$ is an asymmetric factor and describes relaxation properties asymmetric around relaxation frequency

$$ \varepsilon(\omega) = 1 - \frac{\omega_p^2}{\omega^2 + j \omega \gamma} =  1 - \frac{\omega_p^2}{\omega^2 + j \omega \gamma} +  1 - \frac{\omega_p^2}{\omega^2 + j \omega \gamma}  $$

## Arbitrarily-accurate Models

The arbitrarily-accurate (albeit, perhaps unphysical) fits to experimental optical data can be obtained.

- multi-pole Debye model with static conductivity
  $$\varepsilon (\omega) = \varepsilon_\infty  + \sum \limits_i^N \frac{\Delta\varepsilon_i}{1 + j \omega \tau_i} + \frac{\sigma_s}{j \omega \varepsilon_0}$$
  where
  - $\sigma_s$ the static conductivity
  - $\Delta\varepsilon_i$ is the change in permittivity due to the ith dispersion
- Drude model in combination with multiple Lorentz oscillator poles
  $$\varepsilon (\omega) = \varepsilon_\infty  + \sum \limits_N \frac{\Delta\varepsilon_p \omega_p^2}{\omega_p^2 - \omega(\omega + j 2\delta)}$$
- Interpolated tabulated data


## Nonlocal Materials

Spatially-nonlocal effects arise when electrons are confined to small regions of space. Examples include junctions between structures, the areas around the edges of small structures, etc. These can significantly modify the electromagnetic fields, relative to classical (local) effects.


In [ ]:


# Permittivity
# Electric susceptibility

abstract type AbstractPermittivity end
abstract type ConstantPermittivity <: AbstractPermittivity end
abstract type TablePermittivity <: AbstractPermittivity end


# # only real values
# struct DielectricConstant <: ConstantPermittivity
#     eps::Float64
# end
# permittivity(e::DielectricConstant) = e.eps
# permittivity(e::DielectricConstant, lambda) = permittivity(e)
#
# # complex (dielectric + absorption)
# struct PermittivityConstant <: ConstantPermittivity
#     eps::ComplexF64
# end
# permittivity(e::PermittivityConstant) = e.eps
# permittivity(e::PermittivityConstant, lambda) = permittivity(e)


# complex (real or complex)
struct Permittivity{T<:Number} <: ConstantPermittivity
    eps::T
end

permittivity(e::Permittivity) = e.eps
permittivity(e::Permittivity, lambda) = permittivity(e)

struct DrudeModel

end

# Interpolated frequency dependent permitivirty
struct PermittivityTable <: TablePermittivity
    lambda::Vector{Float64}
    eps::Vector{ComplexF64}
    itp_real::Spline1D
    itp_imag::Spline1D
    function PermittivityTable(lambda::Vector{Float64}, eps::Vector{ComplexF64})

        inds = sortperm(lambda)
        # itp_real = interpolate((lambda[inds],), real(eps[inds]), Gridded(Linear()))
        # itp_imag = interpolate((lambda[inds],), imag(eps[inds]), Gridded(Linear()))

        itp_real = Spline1D(lambda[inds], real(eps[inds]))
        itp_imag = Spline1D(lambda[inds], imag(eps[inds]))

        new(lambda, eps, itp_real, itp_imag)
    end
end

permittivity(e::PermittivityTable, lambda) = e.itp_real(lambda) + e.itp_imag(lambda) * im

# TODO: proper broadcasting
reflective_index(e::AbstractPermittivity, lambda) = sqrt(permittivity(e, lambda))


# Constant permitivity

In [ ]:
m1 = Permittivity(2.)
m1, isapprox(permittivity(m1), permittivity(m1, 23523.235))

In [ ]:
m2 = Permittivity(-0.09850 + 1.9392im)
m2, isapprox(permittivity(m2), -0.09850 + 1.9392im)

# Wavelength dependent permitivity of gold

In [ ]:
# Gold, evaporated (Johnson & Christy 1972, PRB 6, 4370)
# wave(um)
eps = [
    -189.0 + 25.36im,
    -125.4 + 12.56im,
    -90.43 + 8.19im,
    -66.22 + 5.70im,
    -51.05 + 3.86im,
    -40.27 + 2.79im,
    -32.04 + 1.93im,
    -25.81 + 1.63im,
    -20.61 + 1.27im,
    -16.82 + 1.07im,
    -13.65 + 1.04im,
    -10.66 + 1.37im,
    -8.11 + 1.66im,
    -5.84 + 2.11im,
    -3.95 + 2.58im,
    -2.28 + 3.81im,
    -1.70 + 4.84im,
    -1.76 + 5.28im,
    -1.69 + 5.65im,
    -1.70 + 5.72im,
    -1.65 + 5.74im,
    -1.60 + 5.64im,
    -1.40 + 5.61im,
    -1.23 + 5.60im,
    -1.31 + 5.54im,
    -1.36 + 5.57im,
    -1.23 + 5.85im,
    -1.24 + 5.79im,
    -1.23 + 5.78im,
    -1.31 + 5.60im,
    -1.33 + 5.49im,
    -1.37 + 5.28im,
    -1.35 + 4.98im,
    -1.24 + 4.72im,
    -1.08 + 4.49im,
    -0.89 + 4.34im,
    -0.74 + 4.16im,
    -0.62 + 4.06im,
    -0.55 + 3.89im,
    -0.42 + 3.83im,
    -0.35 + 3.71im,
    -0.23 + 3.61im,
    -0.13 + 3.51im,
    -0.01 + 3.39im,
    0.14 + 3.40im,
    0.20 + 3.33im,
    0.29 + 3.29im,
    0.30 + 3.18im,
    0.23 + 3.04im
]

lambda = [
    1.937,
    1.610,
    1.393,
    1.216,
    1.088,
    0.9840,
    0.8920,
    0.8211,
    0.7560,
    0.7045,
    0.6595,
    0.6168,
    0.5821,
    0.5486,
    0.5209,
    0.4959,
    0.4714,
    0.4509,
    0.4305,
    0.4133,
    0.3974,
    0.3815,
    0.3679,
    0.3542,
    0.3425,
    0.3315,
    0.3204,
    0.3107,
    0.3009,
    0.2924,
    0.2844,
    0.2761,
    0.2689,
    0.2616,
    0.2551,
    0.2490,
    0.2426,
    0.2371,
    0.2313,
    0.2262,
    0.2214,
    0.2164,
    0.2119,
    0.2073,
    0.2033,
    0.1993,
    0.1953,
    0.1916,
    0.1879
]



In [ ]:
m = PermittivityTable(lambda, eps)

l = LinRange(minimum(lambda), maximum(lambda), 1000)
# l = LinRange(0., maximum(lambda), 1000)
e = permittivity(m, l)
size(e)

In [ ]:
# scatter(lambda, real(eps), xaxis=:log)
scatter(lambda, real(eps))
plot!(l, real(e))

In [ ]:
# scatter(lambda, imag(eps), xaxis=:log)
scatter(lambda, imag(eps))
plot!(l, imag(e))